In [1]:
pip install pandas pyqt5 pymssql sqlalchemy

^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install pyodbc

In [2]:
import sys
import pandas as pd
import re
from datetime import datetime, timedelta
from PyQt5.QtWidgets import (QApplication, QMainWindow, QWidget, QVBoxLayout, 
                              QHBoxLayout, QTextEdit, QPushButton, QTableWidget, 
                             QTableWidgetItem, QLabel, QSplitter, QProgressBar,
                             QMessageBox, QHeaderView)
from PyQt5.QtCore import Qt, QThread, pyqtSignal
from PyQt5.QtGui import QFont
from sqlalchemy import create_engine, text
import warnings
warnings.filterwarnings('ignore')

class DatabaseConnection:
    """Database connection manager for SQL Server using Windows Authentication"""
    
    def __init__(self):
        self.server = "."
        self.database = "AttDB"
        self.engine = None
        
    def connect(self):
        """Establish database connection using Windows Authentication"""
        try:
        # Alternative simple connection string
            connection_uri = "mssql+pyodbc://@./AttDB?driver=SQL+Server&trusted_connection=yes"
        
            self.engine = create_engine(connection_uri)
        
            # Test the connection
            with self.engine.connect() as conn:
                result = conn.execute(text("SELECT 1 as test"))
            print("✅ Database connection test passed!")
        
            return True
        
        except Exception as e:
            print(f"❌ Database connection failed: {e}")
            return False
    
    def execute_query(self, sql_query):
        """Execute SQL query and return DataFrame"""
        try:
            # Use text() to handle SQL queries properly
            with self.engine.connect() as conn:
                df = pd.read_sql(text(sql_query), conn)
            return df
        except Exception as e:
            print(f"❌ Query execution error: {e}")
            print(f"📝 Problematic SQL: {sql_query}")
            # Return empty DataFrame with error info
            return pd.DataFrame()
    
    def close(self):
        """Close database connections"""
        if self.engine:
            self.engine.dispose()

class QueryWorker(QThread):
    """Worker thread for executing queries"""
    finished = pyqtSignal(object)
    error = pyqtSignal(str)
    
    def __init__(self, sql_generator, query):
        super().__init__()
        self.sql_generator = sql_generator
        self.query = query
    
    def run(self):
        try:
            result = self.sql_generator.execute_query(self.query)
            self.finished.emit(result)
        except Exception as e:
            self.error.emit(str(e))

class TextToSQLGenerator:
    def __init__(self, db_connection):
        self.db_conn = db_connection

    def parse_natural_language(self, query):
        """Parse natural language query and convert to SQL"""
        query_lower = query.lower().strip()

        # Extract conditions and date range
        conditions = self._extract_conditions(query)
        date_range = self._extract_date_range(query)

        # Enhanced query type detection with multiple keywords
        query_text = ' '.join(query_lower.split())

        # Check for proxy/attendance data
        if any(word in query_text for word in ['attendance', 'proxy', 'present', 'time in', 'time out', 'punch']):
            return self._handle_attendance_query(query, date_range, conditions)

        # Check for absent/unattended data
        elif any(word in query_text for word in ['absent', 'absentee', 'unattended', 'allow off', 'allowed off']):
            return self._handle_absent_query(query, date_range, conditions)

        # Check for leave/off data
        elif any(word in query_text for word in ['leave', 'off', 'vacation', 'holiday']):
            return self._handle_leave_query(query, date_range, conditions)

        # Check for duty/job data
        elif any(word in query_text for word in ['duty', 'duties', 'job', 'jobs', 'assignment', 'shift']):
            return self._handle_duty_query(query, date_range, conditions)

        # Check for employee info (general data)
        elif any(word in query_text for word in ['employee', 'staff', 'user', 'info', 'information', 'data of', 'provide', 'give', 'show']):
            if conditions or date_range:
                if date_range:
                    return self._handle_attendance_query(query, date_range, conditions)
                else:
                    return self._handle_employee_query(query, date_range, conditions)
            else:
                return self._handle_employee_query(query, date_range, conditions)

        else:
            return self._handle_general_query(query, date_range, conditions)

    def _extract_conditions(self, query):
        """Extract specific conditions from the query"""
        conditions = []
        query_lower = query.lower().strip()

        # User ID condition
        user_id_match = re.search(r'userid\s*[=]?\s*(\d+)', query_lower, re.IGNORECASE)
        if user_id_match:
            conditions.append(f"cu.UserID = {user_id_match.group(1)}")

        # Direct numeric ID pattern
        direct_id_match = re.search(r'(?:^|\s)(\d{3,})(?:\s|$)', query_lower)
        if direct_id_match and not any(word in query_lower for word in ['from', 'to', 'between']):
            potential_id = direct_id_match.group(1)
            if len(potential_id) >= 3:
                conditions.append(f"cu.UserID = {potential_id}")

        # Enhanced username pattern matching
        name_patterns = [
            r'username\s*[=]?\s*([^\s,\d]+(?:\s+[^\s,\d]+)*)',
            r'user\s*name\s*[=]?\s*([^\s,\d]+(?:\s+[^\s,\d]+)*)',
            r'name\s*[=]?\s*([^\s,\d]+(?:\s+[^\s,\d]+)*)',
            r'of\s+([^\s,\d]+(?:\s+[^\s,\d]+)*)',
            r'for\s+([^\s,\d]+(?:\s+[^\s,\d]+)*)',
            r'data\s+of\s+([^\s,\d]+(?:\s+[^\s,\d]+)*)',
            r'info\s+of\s+([^\s,\d]+(?:\s+[^\s,\d]+)*)',
            r'provide\s+([^\s,\d]+(?:\s+[^\s,\d]+)*)',
            r'give\s+([^\s,\d]+(?:\s+[^\s,\d]+)*)',
            r'show\s+([^\s,\d]+(?:\s+[^\s,\d]+)*)',
            r'([^\s,\d]+(?:\s+[^\s,\d]+)*)(?=\s+(?:data|info|attendance|proxy|absent|leave|duty))',
        ]

        username_found = False
        for pattern in name_patterns:
            match = re.search(pattern, query_lower, re.IGNORECASE)
            if match:
                username = match.group(1).strip().title()
                stop_words = ['attendance', 'data', 'duty', 'leave', 'absent', 'from', 'to', 'between',
                             'and', 'the', 'for', 'of', 'proxy', 'unattended', 'allowed', 'off',
                             'provide', 'give', 'show', 'info', 'jobs', 'all', 'me']

                if (any(c.isalpha() for c in username) and
                    username.lower() not in stop_words and
                    not all(word in stop_words for word in username.lower().split())):

                    conditions.append(f"cu.UserName LIKE '%{username}%'")
                    username_found = True
                    break

        # Department condition
        dept_match = re.search(r'department\s*[=]?\s*([^\s,]+)', query_lower, re.IGNORECASE)
        if dept_match:
            conditions.append(f"cu.Department LIKE '%{dept_match.group(1).title()}%'")

        return conditions

    def _extract_date_range(self, query):
        """Extract date range from natural language query"""
        query_lower = query.lower()

        patterns = [
            r'from\s+(\d{1,2}(?:st|nd|rd|th)?\s+\w+\s+\d{4})\s+to\s+(\d{1,2}(?:st|nd|rd|th)?\s+\w+\s+\d{4})',
            r'from\s+(\d{1,2}[-/]\d{1,2}[-/]\d{4})\s+to\s+(\d{1,2}[-/]\d{1,2}[-/]\d{4})',
            r'between\s+(\d{1,2}(?:st|nd|rd|th)?\s+\w+\s+\d{4})\s+and\s+(\d{1,2}(?:st|nd|rd|th)?\s+\w+\s+\d{4})',
            r'between\s+(\d{1,2}[-/]\d{1,2}[-/]\d{4})\s+and\s+(\d{1,2}[-/]\d{1,2}[-/]\d{4})',
            r'in between\s+(\d{1,2}(?:st|nd|rd|th)?\s+\w+\s+\d{4})\s*-\s*(\d{1,2}(?:st|nd|rd|th)?\s+\w+\s+\d{4})',
            r'in between\s+(\d{1,2}[-/]\d{1,2}[-/]\d{4})\s*-\s*(\d{1,2}[-/]\d{1,2}[-/]\d{4})',
            r'(\d{1,2}(?:st|nd|rd|th)?\s+\w+\s+\d{4})\s+to\s+(\d{1,2}(?:st|nd|rd|th)?\s+\w+\s+\d{4})',
            r'(\d{1,2}[-/]\d{1,2}[-/]\d{4})\s+to\s+(\d{1,2}[-/]\d{1,2}[-/]\d{4})',
            r'(\d{1,2}(?:st|nd|rd|th)?\s+\w+\s+\d{4})\s*-\s*(\d{1,2}(?:st|nd|rd|th)?\s+\w+\s+\d{4})',
            r'(\d{1,2}[-/]\d{1,2}[-/]\d{4})\s*-\s*(\d{1,2}[-/]\d{1,2}[-/]\d{4})',
            r'(\d{1,2}(?:st|nd|rd|th)?\s+\w+\s+\d{4})\s+(\d{1,2}(?:st|nd|rd|th)?\s+\w+\s+\d{4})',
        ]

        for pattern in patterns:
            match = re.search(pattern, query_lower)
            if match:
                date1 = self._parse_date_string(match.group(1))
                date2 = self._parse_date_string(match.group(2))
                if date1 and date2:
                    return [date1, date2]

        # YYYY-MM-DD format
        ymd_pattern = r'(\d{4}-\d{2}-\d{2})'
        dates = re.findall(ymd_pattern, query)
        if len(dates) >= 2:
            return [dates[0], dates[1]]

        return None

    def _parse_date_string(self, date_str):
        """Parse natural language date strings"""
        try:
            date_str = re.sub(r'(\d)(st|nd|rd|th)', r'\1', date_str.strip())

            formats = [
                '%d %B %Y',
                '%d %b %Y',
                '%B %d %Y',
                '%b %d %Y',
                '%Y-%m-%d',
                '%d-%m-%Y',
                '%m/%d/%Y',
            ]

            for fmt in formats:
                try:
                    dt = datetime.strptime(date_str, fmt)
                    return dt.strftime('%Y-%m-%d')
                except ValueError:
                    continue
            return None
        except:
            return None

    def _handle_attendance_query(self, query, date_range, conditions):
        """Handle attendance/proxy related queries"""
        where_parts = []
        if date_range:
            where_parts.append(f"va.WorkDay BETWEEN '{date_range[0]}' AND '{date_range[1]}'")
        if conditions:
            where_parts.extend(conditions)

        where_clause = "WHERE " + " AND ".join(where_parts) if where_parts else ""

        sql = f"""
        SELECT
            cu.UserID,
            cu.UserName,
            cu.Department,
            va.WorkDay as Date,
            va.TimeIn,
            va.TimeOut,
            va.Morning,
            va.Evening,
            va.Night,
            CASE
                WHEN va.Morning = 1 OR va.Evening = 1 OR va.Night = 1 THEN 'Present'
                ELSE 'Absent'
            END as Status
        FROM vw_UserAttendance va
        LEFT JOIN tbl_ConsolidatedUsers cu ON va.user_id = cu.UserID
        {where_clause}
        ORDER BY va.WorkDay, cu.UserName
        """
        return sql

    def _handle_absent_query(self, query, date_range, conditions):
        """Handle absent/allowoff/unattended queries"""
        action_conditions = []
        query_lower = query.lower()

        if any(word in query_lower for word in ['allow off', 'allowed off']):
            action_conditions.append("da.ActionType = 'AllowOff'")
        else:
            action_conditions.append("da.ActionType = 'Absent'")

        where_parts = []
        if date_range:
            where_parts.append(f"da._Date BETWEEN '{date_range[0]}' AND '{date_range[1]}'")
        if conditions:
            where_parts.extend(conditions)
        if action_conditions:
            where_parts.extend(action_conditions)

        where_clause = "WHERE " + " AND ".join(where_parts) if where_parts else ""

        sql = f"""
        SELECT
            da.UserID,
            da.UserName,
            da.Department,
            da.Designation,
            da.ActionType,
            da._Date as Date
        FROM tbl_DailyAbsentees da
        {where_clause}
        ORDER BY da._Date, da.UserName
        """
        return sql

    def _handle_leave_query(self, query, date_range, conditions):
        """Handle leave/off data queries"""
        where_parts = []
        if date_range:
            where_parts.append(f"(l.DateStart BETWEEN '{date_range[0]}' AND '{date_range[1]}' OR l.DateEnd BETWEEN '{date_range[0]}' AND '{date_range[1]}')")
        if conditions:
            where_parts.extend(conditions)

        where_clause = "WHERE " + " AND ".join(where_parts) if where_parts else ""

        sql = f"""
        SELECT
            cu.UserID,
            cu.UserName,
            cu.Department,
            l.DateStart,
            l.DateEnd,
            l.TotalDays,
            CASE
                WHEN l.DateStart <= GETDATE() AND l.DateEnd >= GETDATE() THEN 'Current Leave'
                WHEN l.DateEnd < GETDATE() THEN 'Past Leave'
                ELSE 'Upcoming Leave'
            END as LeaveStatus
        FROM tbl_Leave l
        JOIN tbl_ConsolidatedUsers cu ON l.UserID = cu.UserID
        {where_clause}
        ORDER BY l.DateStart, cu.UserName
        """
        return sql

    def _handle_duty_query(self, query, date_range, conditions):
        """Handle duty/job/assignment queries"""
        where_parts = []
        if date_range:
            where_parts.append(f"ud.DutyDate BETWEEN '{date_range[0]}' AND '{date_range[1]}'")
        if conditions:
            where_parts.extend(conditions)

        where_clause = "WHERE " + " AND ".join(where_parts) if where_parts else ""

        sql = f"""
        SELECT
            cu.UserID,
            cu.UserName,
            cu.Department,
            d.DutyName,
            ud.DutyDate,
            CASE
                WHEN ud.MorningDuty = 1 THEN 'Morning'
                WHEN ud.AfternoonDuty = 1 THEN 'Afternoon'
                WHEN ud.NightDuty = 1 THEN 'Night'
                ELSE 'Not Specified'
            END as DutyShift,
            CASE
                WHEN ud.MorningDuty = 1 AND ud.AfternoonDuty = 1 AND ud.NightDuty = 1 THEN 'Full Day'
                WHEN ud.MorningDuty = 1 AND ud.AfternoonDuty = 1 THEN 'Morning-Afternoon'
                WHEN ud.AfternoonDuty = 1 AND ud.NightDuty = 1 THEN 'Afternoon-Night'
                WHEN ud.MorningDuty = 1 AND ud.NightDuty = 1 THEN 'Morning-Night'
                ELSE 'Single Shift'
            END as ShiftCombination
        FROM tbl_UserDuties ud
        JOIN tbl_ConsolidatedUsers cu ON ud.UserID = cu.UserID
        JOIN tbl_Duty d ON ud.DutyID = d.DutyID
        {where_clause}
        ORDER BY ud.DutyDate, d.DutyName, cu.UserName
        """
        return sql

    def _handle_employee_query(self, query, date_range, conditions):
        """Handle employee information queries"""
        where_parts = []
        if conditions:
            where_parts.extend(conditions)

        where_clause = "WHERE " + " AND ".join(where_parts) if where_parts else ""

        sql = f"""
        SELECT
            UserID,
            UserName,
            Department,
            emp_type,
            TYPE,
            CASE WHEN Leave = 1 THEN 'On Leave' ELSE 'Active' END as Status,
            nic_date_exp,
            probation_date,
            PrimaryDevice_Name,
            SecondaryDevice_Name
        FROM tbl_ConsolidatedUsers cu
        {where_clause}
        ORDER BY Department, UserName
        """
        return sql

    def _handle_general_query(self, query, date_range, conditions):
        """Handle general queries about the data"""
        where_parts = []
        if date_range:
            where_parts.append(f"va.WorkDay BETWEEN '{date_range[0]}' AND '{date_range[1]}'")
        if conditions:
            where_parts.extend(conditions)

        where_clause = "WHERE " + " AND ".join(where_parts) if where_parts else ""

        sql = f"""
        SELECT
            cu.UserName,
            cu.Department,
            va.WorkDay as Date,
            va.TimeIn,
            va.TimeOut,
            va.Morning,
            va.Evening,
            va.Night
        FROM vw_UserAttendance va
        JOIN tbl_ConsolidatedUsers cu ON va.user_id = cu.UserID
        {where_clause}
        ORDER BY va.WorkDay DESC, cu.UserName
        LIMIT 50
        """
        return sql

    def execute_query(self, natural_language_query):
        """Main method to execute natural language queries"""
        try:
            sql_query = self.parse_natural_language(natural_language_query)
            df = self.db_conn.execute_query(sql_query)
            
            return {
                'success': True,
                'sql_query': sql_query,
                'data': df,
                'row_count': len(df)
            }

        except Exception as e:
            return {
                'success': False,
                'error': str(e),
                'sql_query': None,
                'data': None
            }

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.db_conn = DatabaseConnection()
        self.sql_generator = None
        self.init_ui()
        self.connect_database()
        
    def init_ui(self):
        """Initialize the user interface"""
        self.setWindowTitle("Text-to-SQL Query Generator - Employee Database")
        self.setGeometry(100, 100, 1400, 900)
        
        # Central widget
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        
        # Main layout
        layout = QVBoxLayout(central_widget)
        
        # Title
        title = QLabel("Text-to-SQL Employee Data Query System")
        title.setFont(QFont("Arial", 16, QFont.Bold))
        title.setAlignment(Qt.AlignCenter)
        title.setStyleSheet("padding: 15px; background-color: #2c3e50; color: white; border-radius: 5px;")
        layout.addWidget(title)
        
        # Database connection status
        self.status_label = QLabel("🔴 Database: Not Connected")
        self.status_label.setFont(QFont("Arial", 10))
        self.status_label.setStyleSheet("padding: 5px;")
        layout.addWidget(self.status_label)
        
        # Create splitter for resizable panels
        splitter = QSplitter(Qt.Vertical)
        layout.addWidget(splitter)
        
        # Top panel - Input
        top_panel = QWidget()
        top_layout = QVBoxLayout(top_panel)
        
        # Query input section
        input_label = QLabel("Enter your query in natural language:")
        input_label.setFont(QFont("Arial", 12, QFont.Bold))
        top_layout.addWidget(input_label)
        
        self.query_input = QTextEdit()
        self.query_input.setPlaceholderText(
            "Examples:\n"
            "• 'Show attendance data for Abid Ali from January 1 2025 to January 3 2025'\n"
            "• 'Give me leave data of John'\n"
            "• 'Show duty assignments for user ID 12746'\n"
            "• 'List all absent employees last week'"
        )
        self.query_input.setMaximumHeight(120)
        self.query_input.setStyleSheet("font-size: 12px; padding: 8px;")
        top_layout.addWidget(self.query_input)
        
        # Button layout
        button_layout = QHBoxLayout()
        
        self.execute_btn = QPushButton("Execute Query")
        self.execute_btn.setStyleSheet(
            "QPushButton { background-color: #27ae60; color: white; font-weight: bold; padding: 8px; border-radius: 4px; }"
            "QPushButton:hover { background-color: #219a52; }"
            "QPushButton:disabled { background-color: #95a5a6; }"
        )
        self.execute_btn.clicked.connect(self.execute_query)
        
        self.clear_btn = QPushButton("Clear")
        self.clear_btn.setStyleSheet(
            "QPushButton { background-color: #e74c3c; color: white; font-weight: bold; padding: 8px; border-radius: 4px; }"
            "QPushButton:hover { background-color: #c0392b; }"
        )
        self.clear_btn.clicked.connect(self.clear_all)
        
        button_layout.addWidget(self.execute_btn)
        button_layout.addWidget(self.clear_btn)
        button_layout.addStretch()
        
        top_layout.addLayout(button_layout)
        
        # Progress bar
        self.progress_bar = QProgressBar()
        self.progress_bar.setVisible(False)
        top_layout.addWidget(self.progress_bar)
        
        # SQL query display
        sql_label = QLabel("Generated SQL Query:")
        sql_label.setFont(QFont("Arial", 12, QFont.Bold))
        top_layout.addWidget(sql_label)
        
        self.sql_display = QTextEdit()
        self.sql_display.setMaximumHeight(100)
        self.sql_display.setStyleSheet("font-family: Consolas, monospace; font-size: 11px; background-color: #f8f9fa;")
        self.sql_display.setReadOnly(True)
        top_layout.addWidget(self.sql_display)
        
        # Bottom panel - Results
        bottom_panel = QWidget()
        bottom_layout = QVBoxLayout(bottom_panel)
        
        results_label = QLabel("Query Results:")
        results_label.setFont(QFont("Arial", 12, QFont.Bold))
        bottom_layout.addWidget(results_label)
        
        self.results_table = QTableWidget()
        self.results_table.setStyleSheet("font-size: 11px;")
        bottom_layout.addWidget(self.results_table)
        
        # Results info
        self.results_info = QLabel("No query executed yet")
        self.results_info.setStyleSheet("padding: 5px; background-color: #ecf0f1; border-radius: 3px;")
        bottom_layout.addWidget(self.results_info)
        
        # Add panels to splitter
        splitter.addWidget(top_panel)
        splitter.addWidget(bottom_panel)
        splitter.setSizes([400, 500])
        
    def connect_database(self):
        """Connect to the database"""
        self.status_label.setText("🔄 Connecting to database...")
        QApplication.processEvents()
        
        if self.db_conn.connect():
            self.sql_generator = TextToSQLGenerator(self.db_conn)
            self.status_label.setText("🟢 Database: Connected to AttDB")
            self.execute_btn.setEnabled(True)
        else:
            self.status_label.setText("🔴 Database: Connection Failed")
            self.execute_btn.setEnabled(False)
            QMessageBox.critical(self, "Database Error", 
                               "Failed to connect to database. Please check:\n"
                               "1. SQL Server is running\n"
                               "2. Database 'AttDB' exists\n"
                               "3. Windows Authentication is enabled\n"
                               "4. You have proper permissions")
    
    def execute_query(self):
        """Execute the natural language query"""
        query_text = self.query_input.toPlainText().strip()
        
        if not query_text:
            QMessageBox.warning(self, "Input Error", "Please enter a query.")
            return
        
        # Disable button and show progress
        self.execute_btn.setEnabled(False)
        self.progress_bar.setVisible(True)
        self.progress_bar.setRange(0, 0)  # Indeterminate progress
        
        # Create worker thread
        self.worker = QueryWorker(self.sql_generator, query_text)
        self.worker.finished.connect(self.on_query_finished)
        self.worker.error.connect(self.on_query_error)
        self.worker.start()
    
    def on_query_finished(self, result):
        """Handle query completion"""
        self.execute_btn.setEnabled(True)
        self.progress_bar.setVisible(False)
        
        # Display SQL query
        if result['sql_query']:
            self.sql_display.setPlainText(result['sql_query'])
        
        if result['success']:
            self.display_results(result['data'])
            self.results_info.setText(f"✅ Query executed successfully. Found {result['row_count']} records.")
        else:
            self.results_table.clear()
            self.results_table.setRowCount(0)
            self.results_table.setColumnCount(0)
            self.results_info.setText(f"❌ Error: {result.get('error', 'Unknown error')}")
    
    def on_query_error(self, error_message):
        """Handle query errors"""
        self.execute_btn.setEnabled(True)
        self.progress_bar.setVisible(False)
        self.results_info.setText(f"❌ Error: {error_message}")
        QMessageBox.critical(self, "Query Error", f"An error occurred:\n{error_message}")
    
    def display_results(self, df):
        """Display pandas DataFrame in the table"""
        if df.empty:
            self.results_table.setRowCount(0)
            self.results_table.setColumnCount(0)
            return
        
        # Set table dimensions
        self.results_table.setRowCount(len(df))
        self.results_table.setColumnCount(len(df.columns))
        
        # Set headers
        self.results_table.setHorizontalHeaderLabels(df.columns)
        
        # Populate table
        for row_idx, row_data in df.iterrows():
            for col_idx, value in enumerate(row_data):
                item = QTableWidgetItem(str(value) if pd.notna(value) else "")
                self.results_table.setItem(row_idx, col_idx, item)
        
        # Adjust column widths
        header = self.results_table.horizontalHeader()
        header.setSectionResizeMode(QHeaderView.ResizeToContents)
        
        # Enable sorting
        self.results_table.setSortingEnabled(True)
    
    def clear_all(self):
        """Clear all inputs and results"""
        self.query_input.clear()
        self.sql_display.clear()
        self.results_table.clear()
        self.results_table.setRowCount(0)
        self.results_table.setColumnCount(0)
        self.results_info.setText("No query executed yet")
    
    def closeEvent(self, event):
        """Handle application closure"""
        if hasattr(self, 'db_conn'):
            self.db_conn.close()
        event.accept()

def main():
    app = QApplication(sys.argv)
    
    # Set application style
    app.setStyle('Fusion')
    
    window = MainWindow()
    window.show()
    
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

✅ Database connection test passed!


SystemExit: 0

In [7]:
import pandas as pd
from sqlalchemy import create_engine, text
import pyodbc

def test_connection():
    print("🔍 Testing SQL Server Connection...")
    
    # Try different connection strings
    connection_strings = [
        # Method 1: Simple Windows Authentication
        "DRIVER={SQL Server};SERVER=.;DATABASE=AttDB;Trusted_Connection=yes;",
        
        # Method 2: Alternative format
        "DRIVER={SQL Server Native Client 11.0};SERVER=.;DATABASE=AttDB;Trusted_Connection=yes;",
        
        # Method 3: Another alternative
        "DRIVER={ODBC Driver 17 for SQL Server};SERVER=.;DATABASE=AttDB;Trusted_Connection=yes;",
    ]
    
    for i, conn_str in enumerate(connection_strings, 1):
        print(f"\n🔄 Trying connection method {i}:")
        print(f"Connection string: {conn_str}")
        
        try:
            # Create connection URI for SQLAlchemy
            connection_uri = f"mssql+pyodbc:///?odbc_connect={conn_str.replace(' ', '%20')}"
            
            engine = create_engine(connection_uri)
            with engine.connect() as conn:
                result = conn.execute(text("SELECT @@VERSION as version"))
                version = result.fetchone()
                print(f"✅ Connection successful!")
                print(f"SQL Server Version: {version[0][:100]}...")
                
                # Test if database exists and has tables
                result = conn.execute(text("""
                    SELECT TABLE_NAME 
                    FROM INFORMATION_SCHEMA.TABLES 
                    WHERE TABLE_TYPE = 'BASE TABLE'
                """))
                tables = result.fetchall()
                print(f"📊 Found {len(tables)} tables in database")
                
                if tables:
                    print("First 10 tables:")
                    for table in tables[:10]:
                        print(f"  - {table[0]}")
                return True
                
        except Exception as e:
            print(f"❌ Connection failed: {e}")
    
    return False

def list_available_drivers():
    print("\n🔍 Available ODBC Drivers:")
    drivers = pyodbc.drivers()
    for driver in drivers:
        print(f"  - {driver}")

if __name__ == "__main__":
    list_available_drivers()
    test_connection()


🔍 Available ODBC Drivers:
  - SQL Server
  - ODBC Driver 17 for SQL Server
  - SQL Server Native Client RDA 11.0
  - Microsoft Access Driver (*.mdb, *.accdb)
  - Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)
  - Microsoft Access Text Driver (*.txt, *.csv)
  - Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)
🔍 Testing SQL Server Connection...

🔄 Trying connection method 1:
Connection string: DRIVER={SQL Server};SERVER=.;DATABASE=AttDB;Trusted_Connection=yes;
✅ Connection successful!
SQL Server Version: Microsoft SQL Server 2022 (RTM) - 16.0.1000.6 (X64) 
	Oct  8 2022 05:58:25 
	Copyright (C) 2022 Micr...
📊 Found 22 tables in database
First 10 tables:
  - tbl_Leave
  - Devices
  - SqlToOracle
  - tbl_ConsolidatedUsers
  - tbl_fkcmd_trans
  - tbl_fkcmd_trans_cmd_param
  - tbl_fkcmd_trans_cmd_result
  - tbl_fkcmd_trans_cmd_result_log_data
  - tbl_fkcmd_trans_cmd_result_user_id_list
  - tbl_fkdevice_status
